In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "5")#When we perform a shuffle, Spark outputs 200 shuffle partitions.
print("SparkSession created with name as 'spark'")

####  Create Dataframe

In [ ]:
myRange = spark.range(1000).toDF("number")
myRange.collect()

In [ ]:
data = [("Java", "20000"), ("Python", "100000")] 
spark.createDataFrame(data).collect()

In [7]:
rdd = spark.sparkContext.parallelize(data)
spark.createDataFrame(rdd).collect()     

[Row(_1='Java', _2='20000'), Row(_1='Python', _2='100000')]

In [10]:
df=spark.read.json("../data/people.json")
df.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [18]:
flightData2015 = spark.read.option("inferSchema", "true")\
.option("header", "true").csv("../../../../hadoopedge/bigdata/modules/spark/git/data/SparkTheDefinitiveGuide/flight-data/csv/2015-summary.csv")
#flightData2015.head(3)
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

#### Spark SQL

In [20]:
flightData2015.createOrReplaceTempView("flight_data_2015")
sqlWay = spark.sql("""SELECT DEST_COUNTRY_NAME, count(1) FROM flight_data_2015 GROUP BY DEST_COUNTRY_NAME """)
dataFrameWay = flightData2015.groupBy("DEST_COUNTRY_NAME").count()

In [21]:
sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#118], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#118, 5), ENSURE_REQUIREMENTS, [id=#123]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#118], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#118] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/F:/hadoopedge/bigdata/modules/spark/git/data/SparkTheDefinitiveG..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#118], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#118, 5), ENSURE_REQUIREMENTS, [id=#136]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#118], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#118] Batched: false, DataFilters: [], Format: CSV, Location: InMe

In [22]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [23]:
maxSql = spark.sql("""SELECT DEST_COUNTRY_NAME, sum(count) as destination_total FROM flight_data_2015 GROUP BY DEST_COUNTRY_NAME ORDER BY sum(count) DESC LIMIT 5 """)
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [28]:
from pyspark.sql.functions import desc
flightData2015.groupBy("DEST_COUNTRY_NAME").sum("count").withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total")).limit(5).show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [27]:
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()

NameError: name 'desc' is not defined